# DCF Valuation Model Data Mining

### 0. Download all Necessary Python Libraries and Packages

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install yfinance pandas numpy datetime
!pip install gspread gspread-dataframe
!pip install requests torch transformers matplotlib scikit-learn python-dateutil tqdm

### 1. Retrieve and/or Download the 3 Financial Statements to CSV files

In [ ]:
import yfinance as yf
import pandas as pd

def get_income_statement(ticker_symbol):

    try:

        ticker = yf.Ticker(ticker_symbol)


        income_statement = ticker.financials

        print(f"Successfully retrieved income statement for {ticker_symbol}")
        return income_statement

    except Exception as e:
        print(f"Error retrieving income statement for {ticker_symbol}: {e}")
        return pd.DataFrame()

def get_balance_sheet(ticker_symbol):

    try:

        ticker = yf.Ticker(ticker_symbol)


        balance_sheet = ticker.balance_sheet

        print(f"Successfully retrieved balance sheet for {ticker_symbol}")
        return balance_sheet

    except Exception as e:
        print(f"Error retrieving balance sheet for {ticker_symbol}: {e}")
        return pd.DataFrame()

def get_cash_flow(ticker_symbol):

    try:

        ticker = yf.Ticker(ticker_symbol)

        cash_flow = ticker.cashflow

        print(f"Successfully retrieved cash flow statement for {ticker_symbol}")
        return cash_flow

    except Exception as e:
        print(f"Error retrieving cash flow statement for {ticker_symbol}: {e}")
        return pd.DataFrame()

def download_csv(ticker_symbol, statement_name="all financial statements"):
    if statement_name == "all financial statements":
        download_csv(ticker_symbol, "cash_flow")
        download_csv(ticker_symbol, "balance_sheet")
        download_csv(ticker_symbol, "income_statement")

        print(f"Successfully downloaded {statement_name} for {ticker_symbol} as .csv")

    elif statement_name == "cash_flow":
        get_cash_flow(ticker_symbol).to_csv(f'{ticker_symbol}_{statement_name}.csv')
        print(f"Successfully downloaded {statement_name} for {ticker_symbol} as .csv")

    elif statement_name == "balance_sheet":
        get_balance_sheet(ticker_symbol).to_csv(f'{ticker_symbol}_{statement_name}.csv')
        print(f"Successfully downloaded {statement_name} for {ticker_symbol} as .csv")

    elif statement_name == "income_statement":
        get_income_statement(ticker_symbol).to_csv(f'{ticker_symbol}_{statement_name}.csv')
        print(f"Successfully downloaded {statement_name} for {ticker_symbol} as .csv")

    else:
        print("2nd argument is not a proper financial statement name")

# test output using Qualcomm: "QCOM"
# get_income_statement("QCOM"))
# get_balance_sheet("QCOM")
# get_cash_flow("QCOM")
# download_csv("QCOM")

### 2. Download Equity Risk Premium, Risk-Free Rate, & T.Bond Rate for the last 4 start of months

In [ ]:
from threading import local
# this program doesn't run on an anaconda notebook hosted on my laptop
# but does on google colab
import requests
import pandas as pd
import io

def get_ERPbymonth_xlsx():

    # The direct download URL for the Excel file
    excel_url = "https://pages.stern.nyu.edu/~adamodar/pc/implprem/ERPbymonth.xlsx"

    # The desired local filename
    local_filename = "ERPbymonth.csv"

    # get past the firewall/anti-bot security i think
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:

        response = requests.get(excel_url, headers=headers)
        # code 200 is the status code for successful request
        if response.status_code == 200:

            df = pd.read_excel(io.BytesIO(response.content))
            with open(local_filename, 'wb') as file:

                # clean up the dataframe to only contain what we need
                df = df.set_index("Start of month")
                return df
        else:
            print(f"Failed to download file. Status code: {response.status_code}")
            print(f"Server response: {response.text}")
    except Exception as e:
        print(f"An error occurred: {e}")

def get_ERP():

    try:

        df = get_ERPbymonth_xlsx()

        local_filename = "ERP.csv"
        with open(local_filename, 'wb') as file:
          columns_to_keep = ['ERP (Normalized)']
          ERP = df.loc[:, columns_to_keep].dropna()
          file.write(df.to_csv(index=True).encode('utf-8'))
          print(f"File downloaded successfully to {local_filename}")
          return ERP




    except Exception as e:
        print(f"An error occurred: {e}")

def get_RiskFree_Rate():

    try:

        df = get_ERPbymonth_xlsx()

        local_filename = "RiskFree_Rate.csv"
        with open(local_filename, 'wb') as file:
            columns_to_keep = ['$ Riskfree Rate']
            df = df.loc[:, columns_to_keep].dropna()
            file.write(df.to_csv(index=True).encode('utf-8'))
            print(f"File downloaded successfully to {local_filename}")
            return df

    except Exception as e:
        print(f"An error occurred: {e}")

def get_T_Bond_Rate():

    try:

        df = get_ERPbymonth_xlsx()

        local_filename = "T_Bond_Rate.csv"
        with open(local_filename, 'wb') as file:
            columns_to_keep = ['T.Bond Rate']
            df = df.loc[:, columns_to_keep].dropna()
            file.write(df.to_csv(index=True).encode('utf-8'))
            print(f"File downloaded successfully to {local_filename}")
            return df

    except Exception as e:
        print(f"An error occurred: {e}")

def get_ERP_T_Bond_Rate_and_RiskFree_Rate():

    # The direct download URL for the Excel file
    excel_url = "https://pages.stern.nyu.edu/~adamodar/pc/implprem/ERPbymonth.xlsx"

    # The desired local filename
    local_filename = "ERPbymonth.csv"

    # get past the firewall/anti-bot security i think
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:

        response = requests.get(excel_url, headers=headers)
    # code 200 is the status code for successful request
        if response.status_code == 200:

            df = pd.read_excel(io.BytesIO(response.content))
            with open(local_filename, 'wb') as file:

                # clean up the dataframe to only contain what we need
                df = df.set_index("Start of month")
                columns_to_keep = ['T.Bond Rate', '$ Riskfree Rate', 'ERP (Normalized)']
                df = df.loc[:, columns_to_keep].dropna()
                file.write(df.to_csv(index=True).encode('utf-8'))
                return df

                print(f"File downloaded successfully to {local_filename}")
        else:
            print(f"Failed to download file. Status code: {response.status_code}")
            print(f"Server response: {response.text}")
    except Exception as e:
        print(f"An error occurred: {e}")


# test output (doesn't use a ticker as an argument)
# get_ERPbymonth_xlsx()
# get_ERP_T_Bond_Rate_and_RiskFree_Rate()
# get_T_Bond_Rate()
# get_RiskFree_Rate()
# get_ERP()

### 3. Download Current Price, Historical Price, and Market Cap for Queried Stock Ticker

In [ ]:
import yfinance as yf
from datetime import datetime

def get_historical_price(ticker, date=None):

    try:

        ticker = yf.Ticker(ticker)
        if date == None:
            end_date = datetime.now()
        else:
            end_date = date.to_datetime()

        start_date = end_date.replace(year=end_date.year - 10)

        df = ticker.history(start=start_date, end=end_date, interval="1d")
        columns_to_keep = ["Close"]
        df = df.loc[:, columns_to_keep].dropna()
        print(f"Successfully retrieved historical stock price for {ticker}")
        return df

    except Exception as e:
        print(f"Error retrieving historical stock price for {ticker}: {e}")
        return pd.DataFrame()

def get_last_stock_price(ticker):

    try:
        df = get_historical_price(ticker)
        df = df.iloc[-1, 0]
        print(f"Successfully retrieved last stock price for {ticker}")
        return df
    except Exception as e:
        print(f"Error retrieving last stock price for {ticker}: {e}")
        return pd.DataFrame()

def get_market_cap(ticker_symbol):

    try:
        outstanding_shares = get_income_statement(ticker_symbol).loc["Diluted Average Shares"]
        market_cap = get_last_stock_price(ticker_symbol) * outstanding_shares[0]
        return market_cap

    except Exception as e:
        print(f"Error calculating market cap for {ticker_symbol}: {e}")
        return pd.DataFrame()

# test output using Qualcomm: "QCOM"
# get_market_cap("QCOM")
# print(get_historical_price("QCOM"))
# get_last_stock_price("QCOM")

### 4. Get Pre-Tax Income and Tax Provision

In [ ]:
import pandas as pd

def get_PreTax_Income(ticker_symbol):
    try:
        df = get_income_statement(ticker_symbol)
        df = df.loc["Pretax Income"]
        df = df.iloc[0]

        print(f"Successfully retrieved Pre-Tax Income for {ticker_symbol}")
        return df

    except Exception as e:
        print(f"Error retrieving Pre-Tax Income for {ticker_symbol}: {e}")
        return pd.DataFrame()


def get_Tax_Provision(ticker_symbol):
    try:
        df = get_income_statement(ticker_symbol)
        df = df.loc["Tax Provision"]
        df = df.iloc[0]

        print(f"Successfully retrieved Tax Provision for {ticker_symbol}")
        return df

    except Exception as e:
        print(f"Error retrieving Tax Provision for {ticker_symbol}: {e}")
        return pd.DataFrame()

def get_corporate_tax_rate(ticker_symbol):

    try:

        corporate_tax_rate = get_Tax_Provision(ticker_symbol) / get_PreTax_Income(ticker_symbol)

        print(f"Successfully calculated the corporate tax rate for {ticker_symbol}")
        return corporate_tax_rate

    except Exception as e:
        print(f"Error calculating the corporate tax rate for {ticker_symbol}: {e}")
        return pd.DataFrame()

# test output using Qualcomm: "QCOM"
# get_PreTax_Income("QCOM")
# get_Tax_Provision("QCOM")
# get_corporate_tax_rate("QCOM")

Successfully retrieved income statement for QCOM
Successfully retrieved Pre-Tax Income for QCOM
Successfully retrieved income statement for QCOM
Successfully retrieved Tax Provision for QCOM
Successfully retrieved income statement for QCOM
Successfully retrieved Tax Provision for QCOM
Successfully retrieved income statement for QCOM
Successfully retrieved Pre-Tax Income for QCOM
Successfully calculated the corporate tax rate for QCOM


np.float64(0.02186532507739938)

### 5. Calculate Beta of a Given Stock

In [ ]:
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta

def calculate_beta(return_stock, return_index="VOO",end_date = datetime.today().date()):

    try:

        # Download historical data (last 5 years)
        start_date = end_date.replace(year=end_date.year - 5)
        df = yf.download([return_stock, return_index], start=start_date, end=end_date, interval = "1mo")

        # Extract daily close prices (should be already adjusted)
        stock_prices = df['Close'][return_stock]
        market_prices = df['Close'][return_index]

        # Calculate daily returns (percent change)
        stock_returns = stock_prices.pct_change().dropna()
        market_returns = market_prices.pct_change().dropna()

        # Calculate covariance between the asset returns and the market returns
        covariance = np.cov(stock_returns, market_returns)[0, 1]

        # Calculate the variance of the market returns
        variance_index = np.var(market_returns)

        # Calculate beta
        beta = covariance / variance_index

        print(f"The beta of {return_stock} relative to {return_index} is: {beta:.4f}")

        return beta

    except Exception as e:
        print(f"Error calculating Beta for {return_stock}: {e}")
        return pd.DataFrame()

# test output using Qualcomm: "QCOM"
# calculate_beta("QCOM")

### 6. Calculate Market Value of Debt of a Given Stock

In [ ]:
def get_debt_market_value(ticker_symbol):
    try:
        balance_sheet = get_balance_sheet(ticker_symbol)
        debt_market_value = balance_sheet.loc["Current Debt"][0] + balance_sheet.loc["Long Term Debt"][0]
        print(f"Successfully calculated the Market Value of Debt for {ticker_symbol}")
        return debt_market_value
    except Exception as e:
        print(f"Error calculating Market Value of Debt for {ticker_symbol}: {e}")
        return pd.DataFrame()


# test output using Qualcomm: "QCOM"
# get_debt_market_value("QCOM")

### 7. Calculate cost of equity for a given stock

In [ ]:
def get_cost_of_equity(ticker_symbol):
    try:
        cost_of_equity = get_T_Bond_Rate().iloc[-1][0] + ( calculate_beta(ticker_symbol) * get_ERP().iloc[-1][0] )
        print(f"Successfully calculated the cost of equity for {ticker_symbol}")
        return cost_of_equity
    except Exception as e:
        print(f"Error calculating cost of equity for {ticker_symbol}: {e}")
        return pd.DataFrame()

# test output using Qualcomm: "QCOM"
# print(get_cost_of_equity("QCOM"))

### 8. Calculate cost of debt for a given stock

In [ ]:
def get_cost_of_debt(ticker_symbol):
    try:
        cost_of_debt = get_income_statement(ticker_symbol).loc['Interest Expense'][0] / get_balance_sheet(ticker_symbol).loc['Total Debt'][0]
        print(f"Successfully calculated the cost of debt for {ticker_symbol}")
        return cost_of_debt
    except Exception as e:
        print(f"Error calculating cost of debt for {ticker_symbol}: {e}")
        return pd.DataFrame()

# test output using Qualcomm: "QCOM"
# print(get_cost_of_debt("QCOM"))


### 9. Calculate Weighted Average Cost of Capital (WACC) for a given stock

In [ ]:
def get_WACC(ticker_symbol):
    try:
        cost_of_equity = get_cost_of_equity(ticker_symbol)
        cost_of_debt = get_cost_of_debt(ticker_symbol)
        corporate_tax_rate = get_corporate_tax_rate(ticker_symbol) #fixed to calc correctly, was previously outputting a dataframe with 5 dates of corporate tax rate
        market_cap = get_market_cap(ticker_symbol)
        debt_market_value = get_debt_market_value(ticker_symbol)

        WACC = ( (market_cap / ( market_cap + debt_market_value )) * cost_of_equity +
                ( ( debt_market_value / ( market_cap + debt_market_value ) ) * cost_of_debt * (1 - corporate_tax_rate) ) )
        print(f"Successfully calculated the WACC for {ticker_symbol}")
        return WACC

    except Exception as e:
        print(f"Error calculating WACC for {ticker_symbol}: {e}")
        return pd.DataFrame()

# test output using Qualcomm: "QCOM"
# print(get_WACC("QCOM"))


### 10. Calculate Comparable Analysis Metrics and Push to CSV File

In [ ]:
import yfinance as yf
import pandas as pd

def calculate_metrics(tickers):
    """
    Calculates key financial metrics for a list of stock tickers.
    """
    metrics_data = []

    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            financials = stock.financials
            balance_sheet = stock.balance_sheet

            # Core Data
            market_cap = info.get('marketCap')
            total_debt = info.get('totalDebt')
            cash = info.get('totalCash')
            ev = market_cap + total_debt - cash if all([market_cap, total_debt, cash]) else None
            print(financials)
            # LTM Data
            ltm_revenue = info.get('totalRevenue')
            ltm_ebitda = info.get('ebitda')
            ltm_ebit = financials.loc['EBIT'][0] if 'EBIT' in financials.index else None
            ltm_eps = info.get('trailingEps')
            current_price = info.get('currentPrice')
            print(ltm_ebit)
            # Tangible Book Value
            total_assets = balance_sheet.loc['Total Assets'][0] if 'Total Assets' in balance_sheet.index else 0
            goodwill = balance_sheet.loc['Good Will'][0] if 'Good Will' in balance_sheet.index else 0
            intangibles = balance_sheet.loc['Intangible Assets'][0] if 'Intangible Assets' in balance_sheet.index else 0
            total_liabilities = balance_sheet.loc['Total Liab'][0] if 'Total Liab' in balance_sheet.index else 0
            tangible_book_value = total_assets - goodwill - intangibles - total_liabilities

            # NTM Data
            ntm_eps = info.get('forwardEps')
            # NTM Revenue & EBITDA require analyst data, not easily available for free.
            # We'll use forward P/E as a proxy for NTM calculation.

            # Calculations
            metrics = {
                'Ticker': ticker,
                'Market Cap': market_cap / 1000000000,
                'Enterprise Value (EV)': ev / 1000000000,
                'EV/Revenue (LTM)': ev / ltm_revenue if ev and ltm_revenue else None,
                'EV/EBITDA (LTM)': ev / ltm_ebitda if ev and ltm_ebitda else None,
                'EV/EBIT (LTM)': ev / ltm_ebit if ev and ltm_ebit else None,
                'P/EPS (LTM)': info.get('trailingPE'),
                'P/Tang BV (LTM)': market_cap / tangible_book_value if market_cap and tangible_book_value > 0 else None,
                'P/E (NTM)': info.get('forwardPE')
            }
            metrics_data.append(metrics)
        except Exception as e:
            print(f"Could not retrieve data for {ticker}: {e}")

    df = pd.DataFrame(metrics_data)
    return df.set_index('Ticker')
# Test Using Qualcomm("QCOM") Comparable Companies
# company_tickers = ["QCOM", "2454.TW", "AVGO", "INTC", "NVDA", "005930.KS", "AAPL", "TSM"]
# metrics_df = calculate_metrics(company_tickers)

# pd.options.display.float_format = '{:,.2f}'.format
# metrics_df.to_csv("comparables_analysis.csv")

### 11. Create a Dataframe w/ all Necessary Calculations to Push to Spreadsheet

In [ ]:
import pandas as pd

        # Manually define your index names
        index_names = ['Pretax Income', 'Tax Provision', 'Corporate Tax Rate',
                      'Beta',
                      'Market Value of Equity', 'Market Value of Debt',
                      'Cost of Equity', 'Cost of Debt',
                      'Weighted Average Cost of Capital (WACC)']

        ticker_symbol = "QCOM"

        # Create a dictionary to hold the values
        data_dict = {
            'Value': [get_PreTax_Income(ticker_symbol), get_Tax_Provision(ticker_symbol), get_corporate_tax_rate(ticker_symbol),
                      calculate_beta(ticker_symbol),
                      get_market_cap(ticker_symbol), get_debt_market_value,
                      get_cost_of_equity(ticker_symbol), get_cost_of_debt(ticker_symbol),
                      get_WACC(ticker_symbol)]
        }

        # Create the DataFrame with the manually named index
        df = pd.DataFrame(data_dict, index=index_names)

        print(df)

### 12. Push Values to DCF Valuation Model Master Sheet
 - you need to copy the json secret key file into the folder on the path in the code in the earlier lines

In [ ]:
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials

def push_dcf_values_to_google_sheet(ticker_symbol):
    try:


        scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        creds = Credentials.from_service_account_file('JSON-SECRET-KEY-FILE', scopes=scope)
        client = gspread.authorize(creds)

        spreadsheet_name = 'DCF Valuation Model Master Sheet'  # Replace with your sheet name
        worksheet_name = 'Income Statement'  # Replace with your specific worksheet name

        spreadsheet = client.open(spreadsheet_name)
        worksheet = spreadsheet.worksheet(worksheet_name)

        set_with_dataframe(worksheet, get_income_statement(ticker_symbol).reset_index(names=""))

        print(f"Successfully pushed value(s) to {worksheet_name}")

        # worksheet.update_cell(1, 2, "New Value")

        #######################################################################

        spreadsheet_name = 'DCF Valuation Model Master Sheet'  # Replace with your sheet name
        worksheet_name = 'Balance Sheet'  # Replace with your specific worksheet name

        worksheet = spreadsheet.worksheet(worksheet_name)

        set_with_dataframe(worksheet, get_balance_sheet(ticker_symbol).reset_index(names=""))

        print(f"Successfully pushed value(s) to {worksheet_name}")

        # worksheet.update_cell(1, 2, "New Value")

        #######################################################################

        spreadsheet_name = 'DCF Valuation Model Master Sheet'  # Replace with your sheet name
        worksheet_name = 'Cash Flow'  # Replace with your specific worksheet name

        worksheet = spreadsheet.worksheet(worksheet_name)

        set_with_dataframe(worksheet, get_cash_flow(ticker_symbol).reset_index(names=""))

        print(f"Successfully pushed value(s) to {worksheet_name} Sheet")

        # worksheet.update_cell(1, 2, "New Value")

        #######################################################################

        spreadsheet_name = 'DCF Valuation Model Master Sheet'  # Replace with your sheet name
        worksheet_name = 'ERP, Risk Free Rate, Treasury Bond Rate'  # Replace with your specific worksheet name

        worksheet = spreadsheet.worksheet(worksheet_name)

        set_with_dataframe(worksheet, get_ERP_T_Bond_Rate_and_RiskFree_Rate().reset_index(names=""))

        print(f"Successfully pushed value(s) to {worksheet_name} Sheet")

        # worksheet.update_cell(1, 2, "New Value")

        #######################################################################

        spreadsheet_name = 'DCF Valuation Model Master Sheet'  # Replace with your sheet name
        worksheet_name = 'Python Calculations'  # Replace with your specific worksheet name

        worksheet = spreadsheet.worksheet(worksheet_name)

        # Manually define your index names
        index_names = ['Pretax Income', 'Tax Provision', 'Corporate Tax Rate',
                      'Beta',
                      'Market Value of Equity', 'Market Value of Debt',
                      'Cost of Equity', 'Cost of Debt',
                      'Weighted Average Cost of Capital ']

        # Create a dictionary to hold the values
        data_dict = {
            'Value': [get_PreTax_Income(ticker_symbol), get_Tax_Provision(ticker_symbol), get_corporate_tax_rate(ticker_symbol),
                      calculate_beta(ticker_symbol),
                      get_market_cap(ticker_symbol), get_debt_market_value(ticker_symbol),
                      get_cost_of_equity(ticker_symbol), get_cost_of_debt(ticker_symbol),
                      get_WACC(ticker_symbol)]
        }

        # Create the DataFrame with the manually named index
        df = pd.DataFrame(data_dict, index=index_names)

        set_with_dataframe(worksheet, df.reset_index(names=""))

        #######################################################################

        spreadsheet_name = 'DCF Valuation Model Master Sheet'  # Replace with your sheet name
        worksheet_name = 'Historical Price'  # Replace with your specific worksheet name

        worksheet = spreadsheet.worksheet(worksheet_name)

        df = get_historical_price("QCOM")

        df = df.reset_index(names="")

        df.iloc[:,0] = df.iloc[:,0].dt.date

        set_with_dataframe(worksheet, df)

        print(f"Successfully pushed value(s) to {worksheet_name} Sheet")

        # worksheet.update_cell(1, 2, "New Value")

        #######################################################################

        print(f"Successfully pushed value(s) to {worksheet_name} Sheet")

        #######################################################################

    except Exception as e:
        print(f"Error pushing value(s) to Google Sheets: {e}")

push_dcf_values_to_google_sheet("QCOM")



### X. Analyze Previous Year's News Sentiment For a Given Stock (W.I.P.)

In [ ]:
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
from dateutil.relativedelta import relativedelta
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm import tqdm

# Setup
# IMPORTANT: Replace with your actual Polygon.io API key
POLYGON_API_KEY = "POLYGON-API-SECRET-KEY"
# Specify the stock ticker you want to analyze
TICKER = "QCOM"

def get_sentiment_finbert(text, tokenizer, model):

    # Analyzes the sentiment of a given text using a FinBERT model

    # Returns a sentiment score between -1 (negative) and 1 (positive)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)

    # Probabilities using softmax
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

    # The model outputs probabilities for [positive, negative, neutral]
    positive_prob = probs[0][0].item()
    negative_prob = probs[0][1].item()

    # Calculate a composite sentiment score
    # Score = (Positive Probability - Negative Probability)
    score = positive_prob - negative_prob
    return score

def fetch_all_news(ticker, api_key):
    # Fetches all news articles for the specified ticker
    all_articles = []
    # Define the date range of news articles to scrape
    end_date = datetime.now()
    start_date = end_date - relativedelta(years=1)

    # Format dates for the API request
    date_format = "%Y-%m-%d"
    url = (f"https://api.polygon.io/v2/reference/news?ticker={ticker}"
           f"&published_utc.gte={start_date.strftime(date_format)}"
           f"&published_utc.lte={end_date.strftime(date_format)}"
           f"&limit=1000&apiKey={api_key}")

    print(f"Fetching news for {ticker} from {start_date.strftime(date_format)} to {end_date.strftime(date_format)}...")

    while url:
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
            data = response.json()

            if 'results' in data and data['results']:
                all_articles.extend(data['results'])

            # Check for the next page URL to handle pagination
            url = data.get('next_url')
            if url:
                url += f"&apiKey={api_key}" # Append API key to the next_url

        except requests.exceptions.RequestException as e:
            print(f"Error fetching news: {e}")
            break

    print(f"Found {len(all_articles)} total news articles.")
    return all_articles

def analyze_and_plot_sentiment():
    # Main function to fetch news, analyze sentiment, and plot the results.
    if POLYGON_API_KEY == "YOUR_API_KEY_HERE":
        print("🚨 Error: Please replace 'YOUR_API_KEY_HERE' with your actual Polygon.io API key.")
        return

    # Fetch News Data
    articles = fetch_all_news(TICKER, POLYGON_API_KEY)
    if not articles:
        print("No articles found. Exiting.")
        return

    df = pd.DataFrame(articles)

    # Initialize FinBERT Model
    print("Initializing FinBERT model... (This may take a moment)")
    model_name = "ProsusAI/finbert"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # Analyze Sentiment for each article
    print("Analyzing sentiment of news articles...")
    # Use tqdm for a progress bar as this step can be slow
    tqdm.pandas(desc="Sentiment Analysis")
    df['sentiment_score'] = df['title'].progress_apply(lambda title: get_sentiment_finbert(title, tokenizer, model))

    # Aggregate Data by Month
    # Convert 'published_utc' to datetime objects and handle potential timezone info
    df['published_utc'] = pd.to_datetime(df['published_utc'], utc=True)
    df.set_index('published_utc', inplace=True)

    # Separate scores into positive and negative, setting the others to 0
    df['positive_scores'] = df['sentiment_score'].apply(lambda x: x if x > 0 else 0)
    df['negative_scores'] = df['sentiment_score'].apply(lambda x: x if x < 0 else 0)

    # Resample to get the monthly sum for each category
    monthly_positive_sum = df['positive_scores'].resample('M').sum()
    # Take the absolute value of the summed negative scores
    monthly_negative_sum = df['negative_scores'].resample('M').sum().abs()

    if monthly_positive_sum.empty and monthly_negative_sum.empty:
        print("Could not calculate monthly sentiment. Check if news articles had valid dates.")
        return

    # Graph the Results
    print("Generating sentiment graph...")
    plt.style.use('seaborn-v0_8-darkgrid')
    fig, ax = plt.subplots(figsize=(14, 7))

    # Plot the positive sentiment sum in blue
    ax.plot(monthly_positive_sum.index, monthly_positive_sum.values, marker='o', linestyle='-', color='blue', label='Positive News Sum')

    # Plot the absolute negative sentiment sum in red
    ax.plot(monthly_negative_sum.index, monthly_negative_sum.values, marker='o', linestyle='-', color='red', label='Absolute Negative News Sum')

    # Formatting the plot
    ax.set_title(f'Monthly Positive vs. Negative News Sentiment for {TICKER} (Past Year)', fontsize=16)
    ax.set_ylabel('Absolute Sum of Sentiment Scores', fontsize=12)
    ax.set_xlabel('Month', fontsize=12)
    ax.legend() # Add a legend to identify the lines

    # Improve date formatting on the x-axis
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    fig.autofmt_xdate() # Auto-rotate date labels

    plt.tight_layout()

    # Save the plot to a file
    plot_filename = f"{TICKER}_sentiment_analysis_split.png"
    plt.savefig(plot_filename)
    print(f"✅ Analysis complete! Graph saved as '{plot_filename}'")

    plt.savefig("news_sentiment.png")

    plt.show()

# --- RUN THE ANALYSIS ---
if __name__ == "__main__":
    analyze_and_plot_sentiment()